In [ ]:
"""
Created on Mon Oct 21 16:13 2024

Prepare what's needed for eORCA1 on the ice-shelf side

@author: Clara Burgard
"""

In [1]:
import xarray as xr
import numpy as np
from cdo import Cdo
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from datetime import date

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 2.1.0


In [4]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

REMAP THE ICE SHELF CONCENTRATION

In [5]:
fNEMO_mask = 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'
dom_cfg_path = inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'
mask_file_path = inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc'
grid_file_path = inputpath_interim + 'grid_eORCA1_T.nc'
#grid_file_path = inputpath_raw + 'eORCA1_v4.2_CDO_with_var.nc'

In [ ]:
cdo.selvar('isf_conc', input = mask_file_path, output = inputpath_interim + 'isf_conc_only_2km.nc')

In [ ]:
cdo.remapbil(grid_file_path, input = inputpath_interim + 'isf_conc_only_2km.nc', output = inputpath_interim + 'isf_conc_only_targetgrid.nc')

REMAP THE IMBIE MASK

In [ ]:
cdo.selvar('Iceshelf_extrap', input = mask_file_path, output = inputpath_interim + 'isfmask_only_2km.nc')

In [ ]:
cdo.remapnn(grid_file_path, input = inputpath_interim + 'isfmask_only_2km.nc', output = inputpath_interim + 'isfmask_only_targetgrid.nc')

CREATE MASK_VARIABLE

In [6]:
domain_cfg = xr.open_dataset(dom_cfg_path)
isfmask = xr.open_dataset(inputpath_interim + 'isfmask_only_targetgrid.nc')
isfconc = xr.open_dataset(inputpath_interim + 'isf_conc_only_targetgrid.nc')

In [7]:
ds_isfNEMO = xr.Dataset()

ds_isfNEMO['mask_isf'] = isfmask['Iceshelf_extrap'].where(domain_cfg['isf_draft'] > 0)
ds_isfNEMO['mask_isf'].attrs['standard_name'] = 'mask of all the ice shelves'
ds_isfNEMO['mask_isf'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

#ds_isfNEMO['mask_isf_onlyisfconc1'] = ds_isfNEMO['mask_isf'].where(isfconc['isf_conc'] > 0.9999)


In [8]:
ds_isfNEMO['floating_frac'] = isfconc['isf_conc'] # .where(isfconc['isf_conc'] < 0.9999,1)

CREATE MASK_ISF_OPEN AND MASK_ISF_CLOSED

In [9]:
file_Justine = xr.open_dataset(mask_file_path).drop('quantile')

In [ ]:
for iid in file_Justine.ID:
    print(iid.values,file_Justine['NAME'].sel(ID=iid).values)

In [10]:
ID_open_list = [1,2,21,66,67,117,124,125,127,128]

In [11]:
file_Justine_open = file_Justine.sel(ID=ID_open_list)
file_Justine_closed = file_Justine.drop_sel(ID=ID_open_list)

In [12]:
domain_cfg_closed =  domain_cfg.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    for vv in ['bottom_level','top_level','isf_draft','bathy_metry']:
        domain_cfg_closed[vv] = domain_cfg_closed[vv].where(ds_isfNEMO['mask_isf'].drop_vars(['glamt', 'gphit']) != id_closed.values, 0)

  0%|          | 0/123 [00:00<?, ?it/s]

In [13]:
mask_isf_open = ds_isfNEMO['mask_isf'].copy()
for id_closed in tqdm(file_Justine_closed.ID):
    mask_isf_open = mask_isf_open.where(ds_isfNEMO['mask_isf'].drop_vars(['glamt', 'gphit']) != id_closed.values)

  0%|          | 0/123 [00:00<?, ?it/s]

In [14]:
mask_isf_closed = ds_isfNEMO['mask_isf'].where(np.isnan(mask_isf_open))

In [15]:
# create mask_isf_open and mask_isf_closed
ds_isfNEMO['mask_isf_open'] = mask_isf_open
ds_isfNEMO['mask_isf_open'].attrs['standard_name'] = 'mask of the ice shelves that are open'
ds_isfNEMO['mask_isf_open'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

ds_isfNEMO['mask_isf_closed'] = mask_isf_closed
ds_isfNEMO['mask_isf_closed'].attrs['standard_name'] = 'mask of the ice shelves that are closed'
ds_isfNEMO['mask_isf_closed'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

IDENTIFY FRONT

In [16]:
mask_0_1_2 = ds_isfNEMO['mask_isf'] * 2
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] != 0, 400) # land
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] == 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(domain_cfg['isf_draft'] == 0, 200) # ice shelf

In [17]:
# mask_open2 is equivalent to mask_isf_closed
mask_0_1_2_corr2 = mask_0_1_2.copy()
extrap_domains_closed = isfmask['Iceshelf_extrap'].copy()

for kisf in ID_open_list:
    mask_0_1_2_closed = mask_0_1_2_corr2.where(ds_isfNEMO['mask_isf'] != kisf, 400)
    extrap_domains_closed = extrap_domains_closed.where(extrap_domains_closed != kisf, np.nan)

In [18]:
is_mask0 = mask_isf_open.where(np.isnan(mask_isf_open)==False,0)
# set all ice shelves to 3
mask_front0 = mask_0_1_2_corr2.where((mask_0_1_2_corr2 == 0) | (mask_0_1_2_corr2 == 400), 300).copy()

In [ ]:
isfmask['Iceshelf_extrap'].where((isfmask['Iceshelf_extrap'] == 55) & (mask_isf_closed > 0)).plot()

In [ ]:
isfmask['Iceshelf_extrap'].where((isfmask['Iceshelf_extrap'] == 55) & (mask_isf_closed > 0)).plot()

In [19]:
mask_front = mask_front0.copy()
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
mask_front = mask_front.where((mask_front0.shift(x=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(x=1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=1)-mask_front0)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,extrap_domains_closed)

In [ ]:
mask_front.plot()

For the ice shelves not resolved at all in eORCA1

In [20]:
mask_0_2 = mask_0_1_2.where(mask_0_1_2 != 200, 2)
mask_0_2 = mask_0_2.where(mask_0_2 != 400, 2)

mask_front_new0 = mask_0_2.copy()

In [61]:
extrap_domains_double_closed = extrap_domains_closed.copy()

for kisf in file_Justine_closed.ID:
    if (mask_front == kisf).astype(int).sum() > 0:
        #print(kisf.values)
        extrap_domains_double_closed = extrap_domains_double_closed.where(extrap_domains_double_closed != kisf, np.nan).drop('ID')

In [65]:
(extrap_domains_double_closed == 15).plot()

In [63]:
mask_0_2 = mask_0_1_2.where(mask_0_1_2 != 200, 2)
mask_0_2 = mask_0_2.where(mask_0_2 != 400, 2)

mask_front_new = mask_0_2.copy()

# check all directions and set points at border between ocean and land(300-0) to 500
mask_front_new = mask_front_new.where((mask_front_new0.shift(x=-1)-mask_front_new0) <= 0,5)
mask_front_new = mask_front_new.where((mask_front_new0.shift(x=1)-mask_front_new0) <= 0,5)
mask_front_new = mask_front_new.where((mask_front_new0.shift(y=-1)-mask_front_new0) <= 0,5)
mask_front_new = mask_front_new.where((mask_front_new0.shift(y=1)-mask_front_new0) <= 0,5)
# cut out all front points
mask_front_new = mask_front_new.where((mask_front_new == 5) & (mask_front_new.y < 90) & (mask_0_2 == 0))
## set the ice shelf number
mask_front_new = mask_front_new.where(mask_front_new != 5 ,extrap_domains_double_closed)

In [41]:
# check that it is the right delimitation
((mask_front_new > 0) + mask_0_2).plot(vmin=14,vmax=17)

In [66]:
mask_front_new.plot(vmin=14,vmax=16)

In [40]:
(mask_front_new + extrap_domains_closed).plot()

In [48]:
(extrap_domains_closed + mask_front_new).plot(vmin=15,vmax=17)

In [52]:
(np.isfinite(mask_front_new) & np.isfinite(extrap_domains_closed)).plot()

In [53]:
mask_front_new.where(extrap_domains_closed == 15).plot()

In [24]:
mask_front_all = mask_front.copy()
for iid in file_Justine_closed.ID:
    if (ds_isfNEMO['mask_isf_closed'] == iid).astype(int).sum().values == 0: #ds_isfNEMO['mask_isf_closed']
        print(iid.values, file_Justine_closed['NAME'].sel(ID=iid).values, (mask_front_new == iid).astype(int).sum().values)
        mask_front_all = mask_front_all.combine_first(mask_front_new.where(mask_front_new == iid))

3  0
6  0
10  2
12  0
13 Shirase 1
14 Hannan/Telen/Skallen 11
15 Rayner_Thyer 0
16 Zubchatyy/Porter/Myers 12
17 WilmaRobertDowner 0
18 Edward_VIII 0
20 Utsikkar/Mulebreen/Cirque_Fjord/Hoseason/Rund_Bay 34
22 Publications 1
23 Sorsdal/Hovde/Flatnes/Dalk 6
25 Helen 6
28  0
30  3
31 Vincennes_Bay/Underwood 3
32 Williamson/Whittle/FoxE 6
35  1
36 Voyeykov 1
37  1
39 Frost 1
40 May_Glacier/Morse/Sandford 5
41 Dibble 1
42 WattBay/Zelee/Astrolabe/Liotard/Francais/Marret/Commandant_Charcot//PourquoiPas 10
44 Ainsworth/Fisher 1
46 Deakin/HornBluff 3
48 Andreyev/Slava 3
50 Drury 0
51 Matusevitch 1
52 Suvorov/Gillet/Noll/Paternostro/McLeod 3
54 Chugunov/Barber/Gannutz 1
55 Lillie 1
56 Arneb/Manhaul/Ironside/Quatermain_Point/Moubray/Dennistoun/SmithInlet//Kirkby 11
57 Tucker 0
58 Mandible_Cirque 2
60 Parker/Falkner/Fitzgerald/Wylde/Suter 1
62 Campbell/CapeWashington/Tinker 3
65 Nordenskjold/Marin/HarbordGlacier/Cheetham/GeikieInlet 12
68  1
69 Hamilton 1
70  1
71 Withrow 1
72  1
73 Richter 1
76  1

In [ ]:
(mask_front_all == 101).plot()

In [ ]:
(mask_front * 0 + 1).plot(vmin=0,vmax=2)
(mask_front_all.where(np.isnan(mask_front)) * 0 + 2).plot(vmin=0,vmax=2)

In [ ]:
mask_front_all.plot(vmin=54,vmax=56)

In [ ]:
##### ds_isfNEMO['mask_isf_closed'].plot()

In [ ]:
mask_front_new.plot()

In [ ]:
(isfmask['Iceshelf'] == 55).plot()

In [ ]:
mask_0_1_2_corr2.where(ds_isfNEMO['mask_isf'] == 55).plot()

In [ ]:
mask_front.plot()

In [ ]:
mask_front.plot(vmin=54,vmax=56)

In [ ]:
mask_front_all.where(mask_front_all==55).plot()

MAKE MAPS OF ZMIN AND ZMAX ON THE FRONT

In [ ]:
zmin_front = mask_front_all.copy()
zmax_front = mask_front_all.copy()

for id_closed in tqdm(file_Justine_closed.ID):
    zmin_front = zmin_front.where(mask_front_all != id_closed, file_Justine_closed['z_min'].sel(ID=id_closed)).drop('ID')
    zmax_front = zmax_front.where(mask_front_all != id_closed, file_Justine_closed['z_max'].sel(ID=id_closed)).drop('ID')

In [ ]:
mask_front_all.plot(vmin=62,vmax=64)

In [ ]:
file_Justine_closed['z_min'].plot()
file_Justine_closed['z_perc01'].plot()

In [ ]:
file_Justine_closed['z_max'].plot()
file_Justine_closed['z_perc99'].plot()

DISTRIBUTE MELT AT THE FRONT NORMALISED BY CELL SIZE

In [ ]:
cell_area = domain_cfg['e1t'] * domain_cfg['e2t']
melt_src = 'Adusumilli' # options: 'Adusumilli','Rignot', 'Paolo', 'Davison'
melt_flux = file_Justine_closed['Melt'+melt_src]

In [ ]:
melt_front = mask_front_all.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    cell_area_isf = cell_area.where(mask_front_all == id_closed)
    cell_area_isf_sum = cell_area.where(mask_front_all == id_closed).sum()
    melt_front = melt_front.where(mask_front_all != id_closed, melt_flux.sel(ID=id_closed) * cell_area_isf / cell_area_isf_sum).drop('ID')

In [ ]:
melt_front.sum()

Double-check that the total melt is matched

In [ ]:
melt_front_tot_list = []
for id_closed in tqdm(file_Justine_closed.ID):
    melt_front_tot_list.append(melt_front.where(mask_front_all == id_closed).sum().assign_coords({'ID': id_closed}))
melt_front_tot_xr = xr.concat(melt_front_tot_list, dim='ID')

In [ ]:
zmin_front = zmin_front.where(melt_front > 0)
zmax_front = zmax_front.where(melt_front > 0)
melt_front = melt_front.where(melt_front > 0)

In [ ]:
melt_front_tot_xr

In [ ]:
(file_Justine_closed['MeltAdusumilli'] - melt_front_tot_xr).where((file_Justine_closed['MeltAdusumilli'] - melt_front_tot_xr) > 0, drop=True)

In [ ]:
file_Justine_closed['NAME'].sel(ID=101).load()

In [ ]:
((mask_0_2) + (zmax_front > 0)).plot()

WRITE TO NETCDF

In [ ]:
ds_isfNEMO['z_min'] = zmin_front
ds_isfNEMO['z_min'].attrs['standard_name'] = 'most shallow point of the ice-shelf draft indicated at the ice-shelf front'
ds_isfNEMO['z_min'].attrs['units'] = 'm below sea level'

ds_isfNEMO['z_max'] = zmax_front
ds_isfNEMO['z_max'].attrs['standard_name'] = 'deepest point of the ice-shelf draft indicated at the ice-shelf front'
ds_isfNEMO['z_max'].attrs['units'] = 'm below sea level'

ds_isfNEMO['melt_isf_closed'] = melt_front
ds_isfNEMO['melt_isf_closed'].attrs['standard_name'] = 'melt flux from '+melt_src+' distributed at the ice-shelf front'
ds_isfNEMO['melt_isf_closed'].attrs['units'] = 'Gt/yr'

ds_isfNEMO['ID'].attrs['standard_name'] = 'ice shelf ID'

In [ ]:
ds_isfNEMO.attrs=dict(Source='Based on IMBIE2 (Mask_Iceshelf_IMBIE2_v2.nc)',
                      Compatibility='The masked variables are compatible with '+fNEMO_mask,
                      Creator='C. Burgard and P. Mathiot ('+date.today().strftime("%b-%d-%Y")+')')

In [ ]:
ds_isfNEMO

In [ ]:
ds_isfNEMO.to_netcdf(inputpath_interim + 'masks_for_eORCA1.nc')